In [6]:
import os
os.chdir(r"D:\Desktop\MLOPS\ds_wine_prediction")
print("Current Working Directory:", os.getcwd())

Current Working Directory: D:\Desktop\MLOPS\ds_wine_prediction


In [7]:
os.getcwd()

'D:\\Desktop\\MLOPS\\ds_wine_prediction'

In [8]:
%pwd

'D:\\Desktop\\MLOPS\\ds_wine_prediction'

In [9]:
import os
#from ensure_annotations import ensure_annotations
import sys
from pathlib import Path
from typing import List
sys.path.append(r"D:\Desktop\MLOPS\ds_wine_prediction")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir : Path

In [11]:
from src.datascience.constants import *
from src.datascience.constants import CONFIG_FILE_PATH
from src.datascience.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        

In [12]:

import os
import urllib.request as request
from src.datascience import logger
import zipfile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Download dataset from the source URL if not already present locally.
        """
        try:
            if not os.path.exists(self.config.local_data_file):
                logger.info(f"Downloading file from {self.config.source_URL} ...")
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"File downloaded successfully: {filename}")
                logger.info(f"Download headers: {headers}")
            else:
                logger.info(f"File already exists at {self.config.local_data_file}")
        except Exception as e:
            logger.error("Error during file download")
            raise e

    def extract_zip_file(self):
        """
        Extract the downloaded zip file into the specified directory.
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extraction completed. Files are available at: {unzip_path}")
        except Exception as e:
            logger.error("Error during file extraction")
            raise e


In [13]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2025-09-24 17:15:13,988: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\config\config.yaml]
[2025-09-24 17:15:13,997: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\params.yaml]
[2025-09-24 17:15:14,006: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\schema.yaml]
[2025-09-24 17:15:14,006: INFO: common: Directory created or already exists: artifacts]
[2025-09-24 17:15:14,012: INFO: common: Directory created or already exists: artifacts/data_ingestion]
[2025-09-24 17:15:14,019: INFO: 1015238248: Downloading file from https://raw.githubusercontent.com/krishnaik06/datasets/main/winequality-data.zip ...]
[2025-09-24 17:15:14,731: INFO: 1015238248: File downloaded successfully: artifacts/data_ingestion/data.zip]
[2025-09-24 17:15:14,750: INFO: 1015238248: Download headers: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-s